# **Train and Test LUIS app**

Steps:
- create an Azure LUIS resource
- add the intents and entities
- format the data to be compatible with LUIS
- add the examples
- run a LUIS model training

## Imports and set-up

In [1]:
import os
import time
from pathlib import Path

import pandas as pd
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import (
    ApplicationCreateObject,
    ExampleLabelObject,
    EntityLabelObject,
)

from msrest.authentication import CognitiveServicesCredentials
from tqdm.notebook import tqdm_notebook as tqdm

In [2]:
# App ID = "c10aeb7c-9207-483b-aa41-6a3027beeb2a"

AZURE_LUIS_ENDPOINT = "https://flymeluis-authoring.cognitiveservices.azure.com/"
AZURE_LUIS_KEY = "ae736d99c5984638a8224a596b40b9cd"

AZURE_LUIS_PROJECT_NAME = "flyme-luis"
AZURE_LUIS_PROJECT_VERSION = "0.1"

AZURE_APP_ID = ""

DATA_PATH = Path("../data")
FRAMES_JSON_PATH = Path(DATA_PATH, "raw/frames.json")

In [3]:
## Create the LUIS Application

# instanciate LUIS Authoring Client
client = LUISAuthoringClient(
    AZURE_LUIS_ENDPOINT, CognitiveServicesCredentials(AZURE_LUIS_KEY)
)

# define app basics
appDefinition = ApplicationCreateObject(
    name=AZURE_LUIS_PROJECT_NAME,
    culture="en-us",
)

# create app
app_id = client.apps.add(appDefinition)

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID cd358193-9fd1-4ac1-a40f-a34e78d2e19d


## Add intents and entities

In [4]:
def add_flyme_intents(version_id):
    book_id = client.model.add_intent(
        app_id=app_id, version_id=version_id, name="Book"
    )
    info_id = client.model.add_intent(
        app_id=app_id, version_id=version_id, name="Info"
    )


def add_flyme_prebuilts(version_id, app_id):
    client.model.add_prebuilt(
        app_id=app_id,
        version_id=AZURE_LUIS_PROJECT_VERSION,
        prebuilt_extractor_names=["geographyV2"],
    )
    client.model.add_prebuilt(
        app_id=app_id,
        version_id=AZURE_LUIS_PROJECT_VERSION,
        prebuilt_extractor_names=["datetimeV2"],
    )
    client.model.add_prebuilt(
        app_id=app_id,
        version_id=AZURE_LUIS_PROJECT_VERSION,
        prebuilt_extractor_names=["number"],
    )


def add_flyme_feature_entity(
        client,
        app_id,
        version_id,
        entity_name,
        model_name,):
    entity_id = client.model.add_entity(
        app_id=app_id, version_id=version_id, name=entity_name
    )
    client.features.add_entity_feature(
        app_id=app_id,
        version_id=version_id,
        entity_id=entity_id,
        feature_relation_create_object={
            "model_name": model_name,
        },
    )


def add_flyme_entities():
    add_flyme_feature_entity(
        client,
        app_id,
        AZURE_LUIS_PROJECT_VERSION,
        "or_city",
        "geographyV2")
    
    add_flyme_feature_entity(
        client,
        app_id,
        AZURE_LUIS_PROJECT_VERSION,
        "dst_city",
        "geographyV2")
    
    add_flyme_feature_entity(
        client,
        app_id,
        AZURE_LUIS_PROJECT_VERSION,
        "str_date",
        "geographyV2")
    
    add_flyme_feature_entity(
        client,
        app_id,
        AZURE_LUIS_PROJECT_VERSION,
        "end_date",
        "geographyV2")
    
    add_flyme_feature_entity(
        client,
        app_id,
        AZURE_LUIS_PROJECT_VERSION,
        "budget",
        "number")

In [5]:
add_flyme_intents(AZURE_LUIS_PROJECT_VERSION)

add_flyme_prebuilts(AZURE_LUIS_PROJECT_VERSION, app_id)

add_flyme_entities()

AttributeError: 'FeaturesOperations' object has no attribute 'add_entity_feature'

In [7]:
app_id

'cd358193-9fd1-4ac1-a40f-a34e78d2e19d'

In [6]:
or_city_id = client.model.add_entity(
    app_id=app_id,
    version_id=AZURE_LUIS_PROJECT_VERSION,
    name="or_city"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_PROJECT_VERSION,
    entity_id=or_city_id,
    feature_relation_create_object={
        "model_name": "geographyV2",
    },
)

ErrorResponseException: Operation returned an invalid status code 'Bad Request'

## Format data for LUIS

In [ ]:
def format_data_for_luis(
        json_path,
        batch_size,
        app_id,
        version_id
):
    raw_data = pd.read_json(json_path)

    entities = ["or_city", "dst_city", "str_date", "end_date", "budget"]
    examples = []
    unique_utterances = []

    for turn in tqdm(raw_data["turns"]):
        for frame in turn:
            if frame["author"] == "wizard" or frame["text"] in unique_utterances:
                continue

            unique_utterances.append(frame["text"])

            is_book = False
            labels = []

            for act in frame["labels"]["acts_without_refs"]:
                for arg in act["args"]:
                    if arg["key"] == "intent" and arg["val"] == "book":
                        is_book = True

                    if (
                        arg["key"] in entities
                        and arg["val"] is not None
                        and frame["text"].find(arg["val"]) != -1
                    ):
                        labels.append(
                            EntityLabelObject(
                                entity_name=arg["key"],
                                start_char_index=frame["text"].find(arg["val"]),
                                end_char_index=frame["text"].find(arg["val"])
                                + len(arg["val"]),
                            )
                        )

            if len(entities) > 0:
                examples.append(
                    ExampleLabelObject(
                        text=frame["text"],
                        intent_name="Book" if is_book else "Info",
                        entity_labels=labels,
                    )
                )

    # add the examples in batch
    for index in tqdm(range(0, len(examples), batch_size)):
        client.examples.batch(
            app_id=app_id,
            version_id=version_id,
            example_label_object_array=examples[index : index + batch_size],
        )

In [ ]:
format_data_for_luis(
    FRAMES_JSON_PATH,
    100,
    app_id,
    AZURE_LUIS_PROJECT_VERSION,
)

## Train data

In [ ]:
def train_flyme_data(client, app_id, version_id):
    client.train.train_version(app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION)
    waiting = True
    while waiting:
        info = client.train.get_status(app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION)

        # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
        waiting = any(
            map(
                lambda x: "Queued" == x.details.status or "InProgress" == x.details.status,
                info,
            )
        )
        if waiting:
            print("Waiting 10 seconds for training to complete...")
            time.sleep(10)
        else:
            print("trained")
            waiting = False

In [ ]:
train_flyme_data(
    client,
    app_id,
    AZURE_LUIS_PROJECT_VERSION
)